In [1]:
!pip install ray[tune] tune-sklearn

In [2]:
import torch
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import make_scorer, mean_squared_log_error, mean_absolute_error, mean_absolute_percentage_error,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingRegressor

data, y = torch.load('../input/housing-price/house_price_v2.pkl')
train_data, test_data = data[:len(y)], data[len(y):]

np.random.seed(42)


std = StandardScaler()
model = BaggingRegressor(base_estimator=SVR())
pipe = Pipeline(
    [('std', std),
     ('model', model)]
)

cross_val_score(pipe, train_data, y, cv=10, scoring=make_scorer(mean_squared_error, greater_is_better=False),n_jobs=-1, verbose=1).mean()


In [ ]:
# 超参数搜索
from ray import tune
from ray.tune.sklearn import TuneSearchCV

param = {
    'model__n_estimators': tune.randint(1, 500),
    # 集成器参数，越多模型越复杂
    'model__base_estimator__C': tune.loguniform(1e-2, 100),
    'model__base_estimator__gamma':tune.loguniform(1e-6,1),
    }


tune_search = TuneSearchCV(
    pipe,
    param,
    search_optimization="bayesian",
    n_trials=-1,
    verbose=1,
    return_train_score=True,
    mode='max',
    scoring=make_scorer(mean_squared_error, greater_is_better=False),
    time_budget_s=20,
    error_score=np.nan,
    cv=10
)
tune_search.fit(train_data, y)
print(tune_search.best_params_)

In [ ]:
best_model = tune_search.best_estimator
mean_squared_error(best_model.predict(train_data),y)

In [ ]:
import pandas as pd

best_model = tune_search.best_estimator
y_test = best_model.predict(test_data.values)
result = pd.DataFrame(np.exp(y_test), index=test_data.index, columns=['SalePrice'], )
result.to_csv('RF_SVM.csv')
torch.save(tune_search.best_params_, 'RF_SVM.pkl')